In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from tensorboard.backend.event_processing import event_accumulator
import pickle

In [2]:
def parse_tensorboard(path, scalars):
    """returns a dictionary of pandas dataframes for each requested scalar"""
    ea = event_accumulator.EventAccumulator(
        path,
        size_guidance={event_accumulator.SCALARS: 0},
    )
    _absorb_print = ea.Reload()
    # make sure the scalars are in the event accumulator tags
    # assert all(
    #     s in ea.Tags()["scalars"] for s in scalars
    # ), "some scalars were not found in the event accumulator"
    res = {}
    for s in scalars:
        try:
            res[s] = pd.DataFrame(ea.Scalars(s))
        except:
            continue
    return res

In [3]:
# scalars = [
#     "val/acc_adv_attack",
#     "val/balanced_acc_adv_attack",
#     "val/acc_task",
#     "val/balanced_acc_task",
#     "train/zero_ratio_adv",
#     "train/zero_ratio_task",
#     "val/acc_task_debiased",
#     "val/acc_protected",
#     "val/balanced_acc_task_debiased",
#     "val/balanced_acc_protected"
# ]
# old suffixes for adv attack: adv_attack_unbiased, adv_attack_biased
scalars = [
    "val/acc_task_debiased",
    "val/acc_task",
    "val/balanced_acc_task_debiased",
    "val/balanced_acc_task",
    "val/acc_adv_attack",
    "val/balanced_acc_adv_attack",
    "val/acc_adv_attack_debiased",
    "val/balanced_acc_adv_attack_debiased",
    # "val/acc_adv_attack_unbiased",
    # "val/balanced_acc_adv_attack_unbiased",
    # "val/acc_adv_attack_biased",
    # "val/balanced_acc_adv_attack_biased"
]

In [4]:
folder = "/share/home/lukash/pan16/bertl4/res_age/logs"
experiment_names = set(["-".join(n.split("-")[:-1]) for n in os.listdir(folder)]) # remove seed suffix
results = {}
for n in experiment_names:
    results[n] = []
    for i in range(5):
        n_seed = n + f"-seed{i}"
        filepath = os.path.join(folder, n_seed)
        df = parse_tensorboard(filepath, scalars)
        results[n].append(df)

In [5]:
# with open("results.pkl", "wb") as f:
#     pickle.dump(results, f)

In [6]:
# with open("results.pkl", "rb") as f:
#     results = pickle.load(f)

In [7]:
scalar_stats = {}
for s in scalars:
    scalar_stats[s] = {}
    for k,v in results.items():
        try:
            scalar = np.array([x[s]["value"].max() for x in v if s in x])
        except KeyError:
            continue
        scalar_stats[s][k] = np.array([scalar.mean(), scalar.std()])

/tmp/lukash/ipykernel_33926/1661473811.py:9: RuntimeWarning: Mean of empty slice.
  scalar_stats[s][k] = np.array([scalar.mean(), scalar.std()])
/home/lukash/miniconda3/envs/testenv/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/lukash/miniconda3/envs/testenv/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/lukash/miniconda3/envs/testenv/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/lukash/miniconda3/envs/testenv/lib/python3.9/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [8]:
list(scalar_stats.keys())

['val/acc_task_debiased',
 'val/acc_task',
 'val/balanced_acc_task_debiased',
 'val/balanced_acc_task',
 'val/acc_adv_attack',
 'val/balanced_acc_adv_attack',
 'val/acc_adv_attack_debiased',
 'val/balanced_acc_adv_attack_debiased']

In [9]:
experiment_names

{'adverserial-baseline-bert_uncased_L-4_H-256_A-4-64-2e-05-age',
 'adverserial-diff_pruning_0.05-bert_uncased_L-4_H-256_A-4-64-2e-05-age',
 'adverserial-diff_pruning_0.1-bert_uncased_L-4_H-256_A-4-64-2e-05-age',
 'modular-merged_head-baseline-bert_uncased_L-4_H-256_A-4-64-2e-05-age',
 'modular-merged_head-diff_pruning_0.05-sparse_task-bert_uncased_L-4_H-256_A-4-64-2e-05-age',
 'modular-merged_head-diff_pruning_0.1-sparse_task-bert_uncased_L-4_H-256_A-4-64-2e-05-age',
 'task-baseline-bert_uncased_L-4_H-256_A-4-64-2e-05-age',
 'task-diff_pruning_0.05-bert_uncased_L-4_H-256_A-4-64-2e-05-age',
 'task-diff_pruning_0.1-bert_uncased_L-4_H-256_A-4-64-2e-05-age'}

In [14]:
n = 'modular-merged_head-diff_pruning_0.05-sparse_task-bert_uncased_L-4_H-256_A-4-64-2e-05-age'
# scalars_to_print = ['val/acc_task', 'val/acc_adv_attack']
scalars_to_print = ['val/acc_task_debiased', 'val/acc_adv_attack_debiased']
format_stats = lambda arr: f"{{${arr[0]*100:.1f} \pm {arr[1]*100:.1f}$}}"

res = {k:[_v for _k,_v in v.items() if _k == n][0] for k,v in scalar_stats.items()}
s = " & ".join([format_stats(res[k]) for k in scalars_to_print])
print(s)

{$91.6 \pm 0.6$} & {$43.3 \pm 0.2$}
